In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os

In [ ]:
PINECONE_API_KEY = ""
PINECONE_API_ENV = ""

In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob = "*.pdf",
                    loader_cls = PyPDFLoader)
    
    documents = loader.load()

    return documents

In [ ]:
os.getcwd()

In [5]:
os.chdir("..") # going back a dir

In [ ]:
os.getcwd()

In [8]:
extracted_data = load_pdf("./data/")

In [90]:
# extracted_data

In [9]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 11


In [93]:
# text_chunks

In [11]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
doc = []
n = 0 
#  {"id": "some_id", "values": vector,  "metadata": metadata}
metadata = {"id": 3056, "description": "Networked neural adapter"}

for t in text_chunks:
    d = {"id" : str(n), "values" : embeddings.embed_query(t.page_content),  "metadata" : {"id": str(n), "description": "Meta"}}
    doc.append(d)
    n += 1

In [16]:
print("Length", type(doc))

Length <class 'list'>


In [18]:
# query_result

In [19]:
from pinecone import Pinecone, PodSpec
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "starter-index"

if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

pc.create_index(
  name="starter-index",
  dimension=384,
  metric="cosine",
  spec=PodSpec(
    environment="gcp-starter"
  )
)

In [20]:
from pinecone import Pinecone, PodSpec
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "starter-index"

index = pc.Index(index_name)

In [22]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
doc

In [24]:
for i in doc:
    # print(i,type(i))
    index.upsert(vectors = [i])

In [25]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [26]:
PROMPT = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
chain_type_kwargs = {"prompt" : PROMPT}

In [27]:
llm = CTransformers(model = "model/llama-2-7b-chat.ggmlv3.q4_0.bin", #model saved in "model" folder
                  model_type = "llama",
                  config = {'max_new_tokens' : 512,
                          'temperature' : 0.8})

In [ ]:
#vectorDB using chromaDB
# from langchain_community.document_loaders import TextLoader
# from langchain_community.embeddings.sentence_transformer import (
#     SentenceTransformerEmbeddings,
# )
# from langchain_community.vectorstores import Chroma


# # create the open-source embedding function
# embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# # load it into Chroma
# db = Chroma.from_documents(text_chunks, embedding_function)

# # query it
# query = "What is achne"
# docs = db.similarity_search(query)

# # print results
# print(docs[0].page_content)

and therefore may help reduce the tendency to bruiseeasily. Green leafy vegetables, alfalfa, broccoli, seaweed,and fish liver oils are dietary sources of vitamin K. Othergood foods to eat would be those containingbioflavonoids, such as reddish-blue berries. These canassist in strengthening the connective tissue, which willdecrease the spread of blood and bruising. Zinc and vita-min C supplements are also recommended for this.
Resources
BOOKS


In [ ]:
# query = "What is achne"
# docs = db.similarity_search(query)

# # print results
# print(docs[0].page_content)

heart muscle, it may help to stabilize arrhythmias. It isgentle and appropriate for home use, unlike foxglove(Digitalis purpurea ), an herb whose action on the heart is
too potent for use without supervision by a qualified prac-titioner. Homeopathic practitioners may prescribe reme-dies such as Lachesis and aconite or monkshood ( Aconi-
tum napellus ) to treat mild arrhythmias.
Prognosis
Advances in diagnostic techniques, new drugs, and


In [ ]:
from langchain.vectorstores import Pinecone

text_field = "text"

vectorstore = Pinecone(
    index, embeddings.embed_query, text_field
)

In [29]:
# query = "What are Allergies"
query = "Ticker tape"
docs = vectorstore.similarity_search(query, k=3)

print("Result", docs)

Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.


Result []


In [30]:
qa = RetrievalQA.from_chain_type(
    llm = llm, 
    chain_type = "stuff",
    retriever = vectorstore.as_retriever(search_kwargs={'k': 2}),
    return_source_documents = True, 
    chain_type_kwargs = chain_type_kwargs
    )

In [31]:
while True:
    user_input = input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

c:\Users\bibas\medical-chatbot\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.


Response :  Ticker tape is a type of paper or plastic ribbon used in ceremonial settings, such as parades and celebrations, to release confetti or streamers onto a crowd or area. It is typically made of thin strips of paper or plastic that are wound on a spool and can be unrolled quickly when needed.


Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.


Response :  The Dimensions menu in Unity allows you to manipulate the size and position of objects in your scene. It is accessible through the GameObject menu, or by using the shortcut key Ctrl + Shift + Z (Windows) or Command + Shift + Z (Mac). From there, you can adjust the dimensions of an object by typing a value in the Dimensions field, or by using the slider to change the size of the object. Additionally, you can use the dimension constraints to restrict the size and position of an object within a specific range.


Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.


Response :  I'm doing well, thanks for asking! How about you?
